# Paper Savior with LionAGI and LlamaIndex Vector Index

-- how to do auto explorative research with LionAGI plus RAG using llamaindex Vector Index & embedding 

- [LionAGI](https://github.com/lion-agi/lionagi)
- [LlamaIndex](https://www.llamaindex.ai)

In [1]:
# %pip install lionagi pypdf llama_index

### 1. Build a Vector Index with llama_index

In [3]:
from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SentenceSplitter
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI


loader = SimpleDirectoryReader(input_dir='papers/', required_exts='.pdf')
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
documents = loader.load_data(show_progress=False)

nodes = node_parser.get_nodes_from_documents(documents, show_progress=False)

# set up index object
llm = OpenAI(temperature=0.1, model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)
index1 = VectorStoreIndex(nodes, include_embeddings=True, 
                          service_context=service_context)

# set up query engine
query_engine = index1.as_query_engine(
    include_text=False, response_mode="tree_summarize"
    )

### 2. Write a tool description according to OpenAI schema

In [4]:
import lionagi as li

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_arxiv_papers",
            "description": """
                           Perform a query to a QA bot with access to an 
                           index built with papers from arxiv
                          """,
            "parameters": {
                "type": "object",
                "properties": {
                    "str_or_query_bundle": {
                        "type": "string",
                        "description": "a question to ask the QA bot",
                    }
                },
                "required": ["str_or_query_bundle"],
            },
        }
    }
]

# we will need to register both the function description 
# and actual implementation
tool = li.Tool(func=query_engine.query, parser=lambda x: x.response, schema_=tools[0])

### 3. Research: PROMPTS

#### FORMATS

In [6]:
# a rigidly set up prompt can help make outcome more deterministic
# though any string will work as well. 
system = {
    "persona": "a helpful world-class researcher",
    "requirements": """
              think step by step before returning a clear, precise 
              worded answer with a humble yet confident tone
          """,
    "responsibilities": f"""
              you are asked to help with researching on the topic 
              of {query}
          """,
    "tools": "provided with a QA bot for grounding responses"
}

# similarly, we can pass in any string or dictionary to instruction
# here we are modifying model behavior by telling mdel how to output 
deliver_format1 = {"return required": "yes", "return format": "paragraph"}

deliver_format2 = {"return required": "yes", 
    "return format": { 
        "json_mode": {
            'paper': "paper_name",
            "summary": "...", 
            "research question": "...", 
            "talking points": {
                "point 1": "...",
                "point 2": "...",
                "point 3": "..."
            }}}}
            
function_call = {
    "notice":f"""
        At each task step, identified by step number, you must use the tool 
        at least twice. Notice you are provided with a QA bot as your tool, 
        the bot has access to the {num_papers} papers via a queriable index 
        that takes natural language query and return a natural language 
        answer. You can decide whether to invoke the function call, you will 
        need to ask the bot when there are things need clarification or 
        further information. you provide the query by asking a question, 
        please use the tool as extensively as you can.
       """
    }

# here we create a two step process imitating the steps human would take to 
# perform the research task
instruct1 = {
    "task step": "1", 
    "task name": "read paper abstracts", 
    "task objective": "get initial understanding of the papers of interest", 
    "task description": """
            provided with abstracts of paper, provide a brief summary 
            highlighting the paper core points, the purpose is to extract 
            as much information as possible
          """,
    "deliverable": deliver_format1
}


instruct2 = {
    "task step": "2",
    "task name": "propose research questions and talking points", 
    "task objective": "initial brainstorming", 
    "task description": """
          from the improved understanding of the paper, please propose 
          an interesting, unique and practical research question, 
          support your reasoning. Kept on asking questions if things are 
          not clear. 
        """,
    "deliverable": deliver_format2,
    "function calling": function_call
}

### 4. Research: Setup Workflow

In [7]:
abstracts = """
Abstract—Large language models (LLMs), such as ChatGPT and GPT4, are making new waves in the field of natural language processing and artificial intelligence, due to their emergent ability and generalizability. However, LLMs are black-box models, which often fall short of capturing and accessing factual knowledge. In contrast, Knowledge Graphs (KGs), Wikipedia and Huapu for example, are structured knowledge models that explicitly store rich factual knowledge. KGs can enhance LLMs by providing external knowledge for inference and interpretability. Meanwhile, KGs are difficult to construct and evolving by nature, which challenges the existing methods in KGs to generate new facts and represent unseen knowledge. Therefore, it is complementary to unify LLMs and KGs together and simultaneously leverage their advantages. In this article, we present a forward-looking roadmap for the unification of LLMs and KGs. Our roadmap consists of three general frameworks, namely, 1) KG-enhanced LLMs, which incorporate KGs during the pre-training and inference phases of LLMs, or for the purpose of enhancing understanding of the knowledge learned by LLMs; 2) LLM-augmented KGs, that leverage LLMs for different KG tasks such as embedding, completion, construction, graph-to-text generation, and question answering; and 3) Synergized LLMs + KGs, in which LLMs and KGs play equal roles and work in a mutually beneficial way to enhance both LLMs and KGs for bidirectional reasoning driven by both data and knowledge. We review and summarize existing efforts within these three frameworks in our roadmap and pinpoint their future research directions.
"""

In [8]:
async def read_propose(context, num=5):
    researcher = li.Session(system, dir=dir)
    researcher.register_tools(tool)
    
    await researcher.initiate(instruct1, context=context, temperature=0.7)
    await researcher.auto_followup(instruct2, tools=tools, num=num)
    
    # researcher.messages_to_csv()
    # researcher.log_to_csv()
    return researcher

### 5. Research: Run the workflow

In [9]:
researcher = await li.alcall(abstracts, read_propose)

2024-01-14 13:50:30,363 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-01-14 13:50:45,724 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [10]:
researcher = researcher[0]

In [12]:
for msg in researcher.conversation.messages:
    if msg.role == "assistant":
        print(f"{msg.msg_content}\n")

The provided abstract discusses the integration of Large Language Models (LLMs) like ChatGPT and GPT4 with Knowledge Graphs (KGs), such as Wikipedia and Huapu, to enhance the capabilities of both systems. LLMs are adept at natural language processing but are often criticized for being "black-box" models with limitations in accessing factual knowledge. KGs, on the other hand, store factual knowledge explicitly but are complex to construct and maintain. The paper proposes a roadmap for combining LLMs and KGs to exploit their respective strengths and mitigate their weaknesses. This unification is structured into three frameworks: KG-enhanced LLMs, which integrate KGs into the training and inference stages of LLMs; LLM-augmented KGs, which use LLMs to assist various KG tasks; and Synergized LLMs + KGs, where both systems work together to improve bidirectional reasoning. The abstract highlights existing efforts within these frameworks and suggests future research directions.

{"function_lis

In [14]:
from IPython.display import Markdown
Markdown(researcher.conversation.messages[-1].msg_content)

Based on the understanding gained from the abstract and the additional insights into the challenges of unifying Large Language Models (LLMs) with Knowledge Graphs (KGs), I propose the following research question and talking points:

**Research Question:**
How can we develop a dynamic compression algorithm that selectively integrates KG information into LLM prompts without losing critical information and maintaining coherence, especially when dealing with large action spaces and API limitations?

**Talking Points:**
1. **Point 1:** Addressing the challenge of integrating reasoning and action capabilities within LLMs, such as the ReAct method, and exploring how to expand the input length limits of in-context learning to accommodate complex tasks with large action spaces.
2. **Point 2:** Investigating techniques for prompt compression, like those used in LLMLingua, to maintain essential information and relevance during the compression process, ensuring that the LLM can still effectively utilize the compressed knowledge.
3. **Point 3:** Considering the selective-context method's limitations in information retention, and developing strategies to prevent loss of context or meaning, which is crucial for the precise interlinking of information in KGs.

To support further exploration and clarification on these points, I will use the QA bot to inquire about specific methods and approaches that exist for prompt compression and the interdependence of information within LLMs and KGs. This will help refine the research question and validate the talking points.